In [ ]:
import os
import re
from urllib.parse import urlparse, parse_qs

import pandas as pd
import boto3

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 200)

%matplotlib inline
%load_ext dotenv
%dotenv

In [ ]:
s3 = boto3.client('s3')
bucket = os.environ["OUTPUT_BUCKET"]
prefix = os.environ["OUTPUT_PREFIX"]

def get_df_from_s3_jsonl(bucket: str, key: str) -> pd.DataFrame:
    df_list = []
    for name in s3.list_objects_v2(Bucket=bucket, Prefix=prefix)['Contents']:
        key = name['Key']
        if key.endswith('.jsonl'):
            obj = s3.get_object(Bucket=bucket, Key=key)
            df_list.append(pd.read_json(obj['Body'], lines=True))

    return pd.concat(df_list, ignore_index=True)

In [ ]:
orig_df = get_df_from_s3_jsonl(bucket, prefix)
df = orig_df.copy()

In [ ]:
# df = orig_df.copy()

# df['id'] = df.url.apply(lambda x: urlparse(x).path.replace('/', '').split('-')[-1]).astype(int)
# df.loc[df.id.duplicated(keep=False), ['url', 'id', 'stats_raw']].sort_values('id')

In [ ]:
df = orig_df.copy()

df['id'] = df.url.apply(lambda x: urlparse(x).path.replace('/', '').split('-')[-1]).astype(int)
df = df.drop_duplicates(subset=["id"], keep="first")

num_only = re.compile(r'\d+')
df['is_project'] = df.project_name.notna()
df['price'] = df.price_raw.apply(lambda x: "".join(re.findall(num_only, x))).astype(int)
df['price_per_m2'] = df.price_per_m2_raw.apply(lambda x: "".join(re.findall(num_only, x))).astype(int)

details_df = pd.json_normalize(df['details_raw'])
stats_df = pd.json_normalize(df['stats_raw'])
address_link = df['precise_address_link']

df = df.drop([
    'details_raw',
    'stats_raw',
    'precise_address_link',
    'decription',
    'price_raw',
    'price_per_m2_raw'
], axis=1)

In [ ]:
def extract_lon_lat(precise_address_link: str):
    result = {"lon": None, "lat": None}

    if precise_address_link:

        query = parse_qs(urlparse(precise_address_link).query)
        coords = query.get('query', [None])[0]

        if coords:
            result['lon'], result['lat'] = coords.split(",")

    return pd.Series(result)

lon_lat_df = address_link.apply(extract_lon_lat)

In [ ]:
details_df = details_df.rename(columns=lambda col: col.strip().lower().replace(" ", "_").replace(".", ""))
# Cast ints
for col in ['house_no', 'flat_no', 'number_of_rooms', 'floor', 'no_of_floors', 'build_year']:
    details_df[col] = details_df[col].astype(int, errors='ignore')

# Clean and cast float
details_df['area'] = details_df['area'].apply(lambda x: float(x.replace(" m²", "").replace(",", ".")) if pd.notna(x) else None)

def make_booleans_from_list_of_features(item, features):
    match item:
        case list():
            return pd.Series({feature: feature in item for feature in features})
        case str():
            return pd.Series({feature: feature == item for feature in features})
        case _ if pd.isna(item):
            return pd.Series({feature: False for feature in features})

features_df_list = []
for col in ['description', 'additional_premises', 'additional_equipment', 'security']:
    features = details_df[col].dropna().explode().unique()
    features_df = details_df[col].apply(lambda x: make_booleans_from_list_of_features(x, features))
    features_df = features_df.rename(columns=lambda c: c.strip().lower().replace(" ", "_").replace(".", ""))
    features_df = features_df.add_prefix(f"{col}_")
    features_df_list.append(features_df)
    details_df = details_df.drop(columns=[col])

features_df = pd.concat(features_df_list, axis=1)
details_df = pd.concat([details_df, features_df], axis=1)
details_df = details_df.add_prefix("details_")

In [ ]:
stats_df = stats_df.rename(columns=lambda col: col.strip().lower().replace(" ", "_"))

stats_df["created"] = pd.to_datetime(stats_df["created"], format="%Y-%m-%d")
stats_df["edited"] = pd.to_datetime(stats_df["edited"], format="%Y-%m-%d")
stats_df["valid_till"] = pd.to_datetime(stats_df["valid_till"], format="%Y-%m-%d")
stats_df["saved"] = stats_df["saved"].astype(int, errors='ignore')

view_stats_df = stats_df["viewed"].apply(lambda x: pd.Series(x.replace(" (totally/today)", "").split("/"), index=["viewed_total", "viewed_today"]).astype(int, errors='ignore'))
stats_df = pd.concat([stats_df.drop(columns=["viewed"]), view_stats_df], axis=1)

In [ ]:
df = pd.concat([
    df,
    lon_lat_df,
    details_df,
    stats_df,
], axis=1)

df = df.convert_dtypes()

In [ ]:
projects_df = df.loc[df.is_project]
second_hand_df = df.loc[~df.is_project]
projects_df.shape, second_hand_df.shape

In [ ]:
projects_agg_df = projects_df.groupby('district').agg({'price': ['median'], 'price_per_m2': ['median', 'count']})
second_hand_agg_df = second_hand_df.groupby('district').agg({'price': ['median'], 'price_per_m2': ['median', 'count']})

In [ ]:
projects_agg_df.columns = ['median_price', 'median_price_per_m2', 'count']
second_hand_agg_df.columns = ['median_price', 'median_price_per_m2', 'count']

In [ ]:
second_hand_agg_gt11_df = second_hand_agg_df.loc[second_hand_agg_df['count'] > 11].sort_values(by='median_price_per_m2', ascending=False)

In [ ]:
second_hand_agg_gt11_df.plot(kind='bar', y='median_price_per_m2', title='Median price per m2 for second hand apartments by district (only districts with more than 11 ads)', figsize=(12, 6))

In [ ]:
second_hand_agg_gt11_df.plot(kind='bar', y='median_price', title='Median price for second hand apartments by district (only districts with more than 11 ads)', figsize=(12, 6))

In [ ]:
second_hand_df.boxplot(column='price_per_m2', by='district', figsize=(16, 8), rot=90)

In [ ]:
second_hand_df.boxplot(column='price', by='district', figsize=(16, 8), rot=90)

In [ ]:
second_hand_df.sort_values(by='price', ascending=False).head(10)